In [1]:
import os
import shutil
import numpy as np

import tensorflow as tf
from tensorflow.keras import backend as keras_backend


gpu_on = True

if gpu_on :
    os.environ['CUDA_VISIBLE_DEVICES'] = '0'
    gpu_devices = tf.config.experimental.list_physical_devices("GPU")
    for device in gpu_devices:
        tf.config.experimental.set_memory_growth(device, True)
else:
    os.environ['CUDA_VISIBLE_DEVICES'] = '-1'
    gpu_devices = tf.config.experimental.list_physical_devices("CPU")

# from tensorflow.keras.mixed_precision import experimental as mixed_precision
# policy = mixed_precision.Policy('mixed_float16')
# mixed_precision.set_policy(policy)

print(gpu_devices)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [2]:
from src.data_loader.classification import BinaryClassifyDataloader as ClassifyDataloader
from tensorflow.keras.applications.inception_v3 import preprocess_input
from glob import glob

task = "classification"
data_set_name = "detect_lvi"
target_size = (512,512)
interpolation = "bilinear"
batch_size = 1
dtype="float32"

train_image_path_list = glob(f"./datasets/{task}/{data_set_name}/train/*/*")
valid_image_path_list = glob(f"./datasets/{task}/{data_set_name}/valid/*/*")
test_image_path_list = glob(f"./datasets/{task}/{data_set_name}/test/*/*")
label_list = os.listdir(f"./datasets/{task}/{data_set_name}/train")

label_to_index_dict = {label:index for index, label in enumerate(label_list)}
index_to_label_dict = {index:label for index, label in enumerate(label_list)}

train_data_loader = ClassifyDataloader(image_path_list=train_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,
                                       shuffle=False,
                                       dtype=dtype
)
valid_data_loader = ClassifyDataloader(image_path_list=valid_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                       
                                       shuffle=False,
                                       dtype=dtype
)
test_data_loader = ClassifyDataloader(image_path_list=test_image_path_list,
                                       label_to_index_dict=label_to_index_dict,
                                       batch_size=batch_size,
                                       preprocess_input=preprocess_input,
                                       target_size=target_size,
                                       interpolation=interpolation,                                      
                                       shuffle=False,
                                      dtype=dtype
)

print(f"train_num: {len(train_image_path_list)}")
print(f"valid_num: {len(valid_image_path_list)}")
print(f"test_num: {len(test_image_path_list)}")

train_num: 8608
valid_num: 1609
test_num: 1469


In [8]:
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.losses import CategoricalCrossentropy, BinaryCrossentropy
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model

import numpy as np
np.random.seed(1337)  # for reproducibility

DROPOUT_RATIO = 0.5

grad_cam = True
transfer_learning = True
epoch_release_frozen = 10
transfer_train_mode = "include_deep_layer"
layer_name_frozen_to = "mixed4"

#  binary_sigmoid, categorical_sigmoid, categorical_softmax
activation = "binary_sigmoid"

# create the base pre-trained model~
base_model = InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    classes=None,
    pooling=None,
    classifier_activation=None
)

if transfer_learning:
    if transfer_train_mode == "dense_only":
        base_model.trainable = False
    elif transfer_train_mode == "include_deep_layer":
        for layer in base_model.layers: 
            layer.trainable = False
            if layer.name == layer_name_frozen_to:
                break

# add a global spatial average pooling layer
x = base_model.output
# (Batch_Size,?)
x = GlobalAveragePooling2D()(x)
x = Dropout(DROPOUT_RATIO)(x)
# let's add a fully-connected layer
# (Batch_Size,1)
x = Dense(1024, activation='relu')(x)
# (Batch_Size,1024)
x = Dropout(DROPOUT_RATIO)(x)

if grad_cam:
    x *= 1e-1
    keras_backend.set_floatx('float64')
    dense_dtype = "float64"
else:
    dense_dtype = "float32"
    
if activation == "binary_sigmoid":
    predictions = Dense(1, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = BinaryCrossentropy(label_smoothing=0.01)
elif activation == "categorical_sigmoid":
    predictions = Dense(2, activation='sigmoid', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)
elif activation == "categorical_softmax":
    predictions = Dense(2, activation='softmax', dtype=dense_dtype)(x)
    loss_function = CategoricalCrossentropy(label_smoothing=0.01)

# this is the model we will train
model = Model(base_model.input, predictions)

# imported_weight_path = "./weights/classification/LVI_detect_classfication/binary_sigmoid/weights_1.0_1.0_InceptionV3_Imagenet_LVI_level2/weights_129_0.0012.hdf5"
# model.load_weights(imported_weight_path)

In [16]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, None, None,  0                                            
__________________________________________________________________________________________________
conv2d_282 (Conv2D)             (None, None, None, 3 864         input_4[0][0]                    
__________________________________________________________________________________________________
batch_normalization_282 (BatchN (None, None, None, 3 96          conv2d_282[0][0]                 
__________________________________________________________________________________________________
activation_282 (Activation)     (None, None, None, 3 0           batch_normalization_282[0][0]    
____________________________________________________________________________________________

In [17]:
from datetime import date

from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import CSVLogger
from tensorflow.keras.callbacks import LambdaCallback
from tensorflow.keras.optimizers import Nadam

today = date.today()

# YY/MM/dd
today_str = today.strftime("%Y/%m/%d")
today_weight_path = f"./weights/{task}/{data_set_name}/{today_str}/" 
today_logs_path = f"./logs/{task}/{data_set_name}/{today_str}/"
os.makedirs(today_weight_path, exist_ok=True)
os.makedirs(today_logs_path, exist_ok=True)
optimizer = Nadam(1e-3, clipnorm=1)

save_c = ModelCheckpoint(
    today_weight_path+"/weights_{epoch:02d}_{loss:.4f}.hdf5",
    monitor='val_loss',
    verbose=0,
    save_best_only=False,
    save_weights_only=True,
    mode='min')


reduceLROnPlat = ReduceLROnPlateau(
    monitor="val_loss",
    factor=0.1,
    patience=10,
    verbose=1,
    mode="auto",
    min_delta=0.0001,
    cooldown=5,
    min_lr=1e-9)
csv_logger = CSVLogger(f'{today_logs_path}/log.csv', append=False, separator=',')


def make_model_trainable(epoch, model, target_epoch):
    if epoch > target_epoch:
        for layer in model.layers:
            layer.trainable = True
            
make_trainable_callback = LambdaCallback(
    on_epoch_begin=lambda epoch,logs: make_model_trainable(epoch, model, target_epoch=epoch_release_frozen)
)

model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])

In [18]:
# start_epoch = 0
# epochs = 200

# model.fit(
#     train_data_loader,
#     validation_data=valid_data_loader,
#     epochs=epochs,
#     callbacks=[reduceLROnPlat, save_c, csv_logger],
#     initial_epoch=start_epoch
# )

# Class Activation Map

In [105]:
import cv2
import os
import numpy as np

def imread(img_path, channel=None):
    img_byte_stream = open(img_path.encode("utf-8"), "rb")
    img_byte_array = bytearray(img_byte_stream.read())
    img_numpy_array = np.asarray(img_byte_array, dtype=np.uint8)
    img_numpy_array = cv2.imdecode(
        img_numpy_array, cv2.IMREAD_UNCHANGED)
    if channel == "rgb":
        img_numpy_array = cv2.cvtColor(
            img_numpy_array, cv2.COLOR_BGR2RGB)
    if len(img_numpy_array.shape) == 2:
        img_numpy_array = np.expand_dims(img_numpy_array, axis=-1)
    return img_numpy_array

from tensorflow import keras
import matplotlib.cm as cm

def get_last_conv_name(model):
    layer_names = [layer.name for layer in model.layers]
    conv_layer_name = [layer_name for layer_name in layer_names if layer_name.find("conv") >= 0]
    last_conv_name = conv_layer_name[-1]
    
    return last_conv_name

def get_img_array(img_path, channel="rgb"):
    
    img_array = imread(img_path, channel="rgb")
    img_array = np.expand_dims(img_array, axis=0)
    
    return img_array

def make_grad_model(model, target_layer_name):
    # Model input: model.input
    # Model output: [last_conv_layer_outputs, model.outputs]
    grad_model = keras.models.Model(
        model.input, [model.get_layer(target_layer_name).output, model.output]
    )
    return grad_model

def make_gradcam_heatmap(img_array, grad_model, pred_index=None):
    # compute
    # 1. last_conv_layer_output
    # 2. class_channel value in prediction
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]

    # compute gradient 
    grads = tape.gradient(class_channel, last_conv_layer_output)

    # compute gradient channel mean 
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    
    # compute matmul(last_conv_layer_output, pooled_grads)
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)

    # For visualization purpose, we will also normalize the heatmap between 0 & 1
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def save_and_display_gradcam(img_array_normalized, heatmap, cam_path="cam.jpg", alpha=0.4, display_cam=True):
    # Load the original image
    img_array = (img_array_normalized + 1) * 127.5
    img_array = img_array.astype('uint8')
    img_array = img_array[0]
    # Rescale heatmap to a range 0-255
    heatmap = np.uint8(255 * heatmap)

    # Use jet colormap to colorize heatmap
    jet = cm.get_cmap("jet")

    # Use RGB values of the colormap
    jet_colors = jet(np.arange(256))[:, :3]
    jet_heatmap = jet_colors[heatmap]

    # Create an image with RGB colorized heatmap
    jet_heatmap = keras.preprocessing.image.array_to_img(jet_heatmap)
    jet_heatmap = jet_heatmap.resize((img_array.shape[1], img_array.shape[0]))
    jet_heatmap = keras.preprocessing.image.img_to_array(jet_heatmap)

    # Superimpose the heatmap on original image
    superimposed_img = jet_heatmap * alpha + img_array
    superimposed_img = keras.preprocessing.image.array_to_img(superimposed_img)

    # Save the superimposed image
    superimposed_img.save(cam_path)

    # Display Grad CAM
    if display_cam:
        display(Image.open(cam_path))


def decode_classify_predictions(pred_array, index_to_label_dict):
    predicted_index = np.sum(np.argmax(preds, axis=1))
    predicted = index_to_label_dict[predicted_index]
    
    return predicted

def decode_binary_classify_predictions(pred_array):
    predicted = np.round(pred_array).astype("int32")
    predicted = predicted.flatten()[0]
    return predicted

def restore_sigmoid_value(y, decrease_ratio=1e1):
    original_x = np.log(y/(1-y))
    restored_x = decrease_ratio*original_x
    new_y = 1/(1+np.exp(-restored_x))
    return new_y


In [ ]:
def remove_folder_png_files(folder_path):
    regxp_files = os.path.join(folder_path, "*.png")
    target_files = glob(regxp_files)
    for target_file in target_files:
        os.remove(target_file)

In [101]:
layer_names = [layer.name for layer in model.layers]
conv_layer_name = [layer_name for layer_name in layer_names if layer_name.find("conv") >= 0]
last_conv_name = conv_layer_name[-1]

print(last_conv_name)

conv2d_375


In [74]:
target_data_loader = test_data_loader

target_data_loader.batch_size = 1

model_predicted = model.predict(target_data_loader)
ground_truth = np.array([label for img_array, label in target_data_loader])

model.evaluate(target_data_loader)

1469/1469 [==============================] - 326s 222ms/step - loss: 0.1767 - accuracy: 0.9973


[0.1767324041790748, 0.9972770592239619]

In [75]:
negative_group_1 = np.where(model_predicted < 0.5)[0]
negative_group_2 = np.where(model_predicted != 0)[0]
negative_confirm = np.where(model_predicted == 0)[0]
negative_not_confirm = np.intersect1d(negative_group_1, negative_group_2)

positive_group_1 = np.where(model_predicted > 0.5)[0]
positive_group_2 = np.where(model_predicted != 1)[0]
postive_confirm = np.where(model_predicted == 1)[0]
postive_not_confirm = np.intersect1d(positive_group_1, positive_group_2)

ground_truth_negative = np.where(ground_truth == 0)[0]
ground_truth_positive = np.where(ground_truth == 1)[0]

true_negative = np.intersect1d(negative_group_1, ground_truth_negative)
true_positive = np.intersect1d(positive_group_1, ground_truth_positive)
false_negative = np.intersect1d(negative_group_1, ground_truth_positive)
false_positive = np.intersect1d(positive_group_1, ground_truth_negative)


In [76]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
769
0
700


In [77]:
print(len(negative_confirm))
print(len(negative_not_confirm))
print(len(postive_confirm))
print(len(postive_not_confirm))

0
769
0
700


In [78]:
print(false_negative)
print(false_positive)

[1234 1395]
[ 32 471]


In [116]:
from matplotlib import pyplot as plt

target_layer_name = "mixed10"
target_cam_folder = "./grad_cam/"
true_negative_folder = f"{target_cam_folder}/true_negative/"
true_positive_folder = f"{target_cam_folder}/true_positive/"
false_negative_folder = f"{target_cam_folder}/false_negative/"
false_positive_folder = f"{target_cam_folder}/false_positive/"

os.makedirs(true_negative_folder, exist_ok=True)
os.makedirs(true_positive_folder, exist_ok=True)
os.makedirs(false_negative_folder, exist_ok=True)
os.makedirs(false_positive_folder, exist_ok=True)

remove_folder_png_files(true_negative_folder)
remove_folder_png_files(true_positive_folder)
remove_folder_png_files(false_negative_folder)
remove_folder_png_files(false_positive_folder)

grad_model = make_grad_model(model, target_layer_name)

for index, (img_array_normalized, label) in enumerate(target_data_loader):
    
    # Print what the top predicted class is
    preds = model.predict(img_array_normalized)
    preds_value = preds.flatten()[0]
    preds_value = restore_sigmoid_value(preds_value, decrease_ratio=1e1)
    predicted_label = decode_binary_classify_predictions(preds)
    label = decode_binary_classify_predictions(label)
    pred_index = None
    # pred_index = label_to_index_dict[predicted_label]

    print(preds)
    print(preds_value)
    print(f"Predicted: {predicted_label}")

    # Generate class activation heatmap
    heatmap = make_gradcam_heatmap(img_array_normalized, grad_model, pred_index=pred_index)
    
    if predicted_label == 0 and label == 0:
        cam_path = f"{true_negative_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 1 and label == 1:
        cam_path = f"{true_positive_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 0 and label == 1:
        cam_path = f"{false_negative_folder}/cam_{index}_{preds_value:.4f}.png"
    if predicted_label == 1 and label == 0:
        cam_path = f"{false_positive_folder}/cam_{index}_{preds_value:.4f}.png"
    
    save_and_display_gradcam(img_array_normalized, heatmap, cam_path=cam_path, display_cam=False)
    

[[0.17763427]]
2.2111465494193163e-07
Predicted: 0
[[0.15612637]]
4.6988146345252175e-08
Predicted: 0
[[0.14576807]]
2.0934668965166156e-08
Predicted: 0
[[0.22594494]]
4.490613475918599e-06
Predicted: 0
[[0.13144295]]
6.300708444081923e-09
Predicted: 0
[[0.17118787]]
1.4130872703023344e-07
Predicted: 0
[[0.13761907]]
1.0710130943496393e-08
Predicted: 0
[[0.1672932]]
1.0711779532181592e-07
Predicted: 0
[[0.31715671]]
0.00046700944946627746
Predicted: 0
[[0.39074048]]
0.01163531057035048
Predicted: 0
[[0.23144941]]
6.1352660045674234e-06
Predicted: 0
[[0.23236632]]
6.459349581292263e-06
Predicted: 0
[[0.19473849]]
6.841453429910626e-07
Predicted: 0
[[0.15073082]]
3.101465731913509e-08
Predicted: 0
[[0.13228437]]
6.781272750829716e-09
Predicted: 0
[[0.09762809]]
2.1973150314954838e-10
Predicted: 0
[[0.20091718]]
1.009839627818521e-06
Predicted: 0
[[0.16910344]]
1.2191428812899984e-07
Predicted: 0
[[0.29763894]]
0.0001867291817832621
Predicted: 0
[[0.26803344]]
4.3346958957712346e-05
Predi

[[0.15091122]]
3.145459682440603e-08
Predicted: 0
[[0.14534706]]
2.0237856084436184e-08
Predicted: 0
[[0.14235979]]
1.5878954734069055e-08
Predicted: 0
[[0.13977874]]
1.2832448641948886e-08
Predicted: 0
[[0.14695702]]
2.3024713967855335e-08
Predicted: 0
[[0.21941991]]
3.0802389577457692e-06
Predicted: 0
[[0.21717972]]
2.7012880349131996e-06
Predicted: 0
[[0.25202031]]
1.8855985692137058e-05
Predicted: 0
[[0.13251705]]
6.920034735077003e-09
Predicted: 0
[[0.21209295]]
1.997613615155934e-06
Predicted: 0
[[0.24711076]]
1.4507725807712255e-05
Predicted: 0
[[0.16129988]]
6.922663475363062e-08
Predicted: 0
[[0.14352974]]
1.7470128373020424e-08
Predicted: 0
[[0.46020523]]
0.16865848601317682
Predicted: 0
[[0.22060603]]
3.3006695615393377e-06
Predicted: 0
[[0.26800022]]
4.327360925816749e-05
Predicted: 0
[[0.11618597]]
1.5414452443395814e-09
Predicted: 0
[[0.23972196]]
9.712914688358714e-06
Predicted: 0
[[0.11715143]]
1.692831773871041e-09
Predicted: 0
[[0.15056924]]
3.06254754720888e-08
Predi

[[0.25128486]]
1.8133812144303676e-05
Predicted: 0
[[0.15572844]]
4.558876795786504e-08
Predicted: 0
[[0.24769688]]
1.4971665531976908e-05
Predicted: 0
[[0.2161375]]
2.540394630640755e-06
Predicted: 0
[[0.12292779]]
2.9251446195530524e-09
Predicted: 0
[[0.17659028]]
2.0582965787838244e-07
Predicted: 0
[[0.0747757]]
1.1888710167708614e-11
Predicted: 0
[[0.13622025]]
9.514514541117011e-09
Predicted: 0
[[0.09859462]]
2.450928014864536e-10
Predicted: 0
[[0.09919322]]
2.6212211090319144e-10
Predicted: 0
[[0.2024664]]
1.1118325038227986e-06
Predicted: 0
[[0.27060396]]
4.9395351835909166e-05
Predicted: 0
[[0.1953902]]
7.13139305872156e-07
Predicted: 0
[[0.29461212]]
0.00016149387047369147
Predicted: 0
[[0.1267663]]
4.15655062579504e-09
Predicted: 0
[[0.15987283]]
6.227248207050041e-08
Predicted: 0
[[0.239682]]
9.691637562509374e-06
Predicted: 0
[[0.23798101]]
8.825943482432117e-06
Predicted: 0
[[0.09207192]]
1.1501582838121234e-10
Predicted: 0
[[0.14635608]]
2.1945228274330184e-08
Predicted: 

[[0.11729861]]
1.7170806649683674e-09
Predicted: 0
[[0.12729134]]
4.358084855765257e-09
Predicted: 0
[[0.24511927]]
1.3031231910988852e-05
Predicted: 0
[[0.10762105]]
6.508475800505396e-10
Predicted: 0
[[0.29691799]]
0.00018039596641646272
Predicted: 0
[[0.09601396]]
1.826949286840472e-10
Predicted: 0
[[0.1680673]]
1.1322707736464254e-07
Predicted: 0
[[0.19280245]]
6.04403254215722e-07
Predicted: 0
[[0.12718287]]
4.3157216358426e-09
Predicted: 0
[[0.27903788]]
7.541790913299901e-05
Predicted: 0
[[0.15159482]]
3.317496844443349e-08
Predicted: 0
[[0.07761588]]
1.779867752602423e-11
Predicted: 0
[[0.07286811]]
8.994140410366316e-12
Predicted: 0
[[0.26916712]]
4.5921836587380306e-05
Predicted: 0
[[0.09449171]]
1.5311317094490737e-10
Predicted: 0
[[0.06604251]]
3.1258255482202006e-12
Predicted: 0
[[0.17014891]]
1.3130766244343357e-07
Predicted: 0
[[0.10127104]]
3.3003911954244596e-10
Predicted: 0
[[0.09676406]]
1.9912631783099922e-10
Predicted: 0
[[0.30197315]]
0.00022954208303528514
Predic

[[0.05322576]]
3.15322305932228e-13
Predicted: 0
[[0.3917772]]
0.012146664258648915
Predicted: 0
[[0.18703255]]
4.1537478040639515e-07
Predicted: 0
[[0.16714466]]
1.059813106456582e-07
Predicted: 0
[[0.21455244]]
2.3129256083118128e-06
Predicted: 0
[[0.22543169]]
4.360643465788248e-06
Predicted: 0
[[0.30325966]]
0.00024396687002091865
Predicted: 0
[[0.17334166]]
1.643497405609722e-07
Predicted: 0
[[0.07939263]]
2.275398991870937e-11
Predicted: 0
[[0.06864275]]
4.7291591611075965e-12
Predicted: 0
[[0.10402125]]
4.4488072870786144e-10
Predicted: 0
[[0.2220237]]
3.583670470097936e-06
Predicted: 0
[[0.12459661]]
3.411770805692811e-09
Predicted: 0
[[0.20547279]]
1.3379960141170465e-06
Predicted: 0
[[0.23287254]]
6.645147874062377e-06
Predicted: 0
[[0.09410051]]
1.4625807816090458e-10
Predicted: 0
[[0.06987881]]
5.728743726782854e-12
Predicted: 0
[[0.10143051]]
3.358686518020592e-10
Predicted: 0
[[0.28491974]]
0.0001008402021158356
Predicted: 0
[[0.06109636]]
1.3612654685328236e-12
Predicted

[[0.94944335]]
0.9999999999998168
Predicted: 1
[[0.99574122]]
1.0
Predicted: 1
[[0.64740696]]
0.9977093022886423
Predicted: 1
[[0.63856151]]
0.9966360210576849
Predicted: 1
[[0.88166388]]
0.9999999981026702
Predicted: 1
[[0.7271706]]
0.9999447255396497
Predicted: 1
[[0.90900314]]
0.9999999998989328
Predicted: 1
[[0.89990973]]
0.9999999997103128
Predicted: 1
[[0.96865356]]
0.9999999999999987
Predicted: 1
[[0.81302607]]
0.9999995862235915
Predicted: 1
[[0.65990544]]
0.9986799021497988
Predicted: 1
[[0.82621862]]
0.9999998305340437
Predicted: 1
[[0.95312876]]
0.9999999999999174
Predicted: 1
[[0.9820379]]
1.0
Predicted: 1
[[0.95590699]]
0.9999999999999565
Predicted: 1
[[0.97637988]]
1.0
Predicted: 1
[[0.94190372]]
0.9999999999992031
Predicted: 1
[[0.98117768]]
1.0
Predicted: 1
[[0.96419742]]
0.9999999999999951
Predicted: 1
[[0.94522919]]
0.9999999999995732
Predicted: 1
[[0.93100956]]
0.9999999999950071
Predicted: 1
[[0.85786719]]
0.9999999844137106
Predicted: 1
[[0.94433115]]
0.99999999999

[[0.95424922]]
0.9999999999999358
Predicted: 1
[[0.96430427]]
0.9999999999999951
Predicted: 1
[[0.9304997]]
0.9999999999945961
Predicted: 1
[[0.91957125]]
0.9999999999738025
Predicted: 1
[[0.9512496]]
0.999999999999875
Predicted: 1
[[0.70576659]]
0.999841423041366
Predicted: 1
[[0.88058281]]
0.9999999978963836
Predicted: 1
[[0.93872837]]
0.9999999999985965
Predicted: 1
[[0.92428948]]
0.9999999999864018
Predicted: 1
[[0.97301213]]
0.9999999999999998
Predicted: 1
[[0.88189863]]
0.9999999981449192
Predicted: 1
[[0.98503495]]
1.0
Predicted: 1
[[0.99145469]]
1.0
Predicted: 1
[[0.88635972]]
0.9999999987998855
Predicted: 1
[[0.98549107]]
1.0
Predicted: 1
[[0.85458808]]
0.9999999796562207
Predicted: 1
[[0.95302355]]
0.9999999999999154
Predicted: 1
[[0.95810552]]
0.9999999999999745
Predicted: 1
[[0.92418513]]
0.9999999999861977
Predicted: 1
[[0.96555797]]
0.9999999999999967
Predicted: 1
[[0.9344967]]
0.9999999999971367
Predicted: 1
[[0.98379066]]
1.0
Predicted: 1
[[0.85986019]]
0.99999998677679

[[0.90628234]]
0.9999999998601767
Predicted: 1
[[0.96460824]]
0.9999999999999956
Predicted: 1
[[0.99063101]]
1.0
Predicted: 1
[[0.98773845]]
1.0
Predicted: 1
[[0.96053089]]
0.9999999999999862
Predicted: 1
[[0.9264909]]
0.9999999999901146
Predicted: 1
[[0.96547831]]
0.9999999999999967
Predicted: 1
[[0.99224169]]
1.0
Predicted: 1
[[0.98062954]]
1.0
Predicted: 1
[[0.97519246]]
0.9999999999999998
Predicted: 1
[[0.99354521]]
1.0
Predicted: 1
[[0.99542543]]
1.0
Predicted: 1
[[0.99181145]]
1.0
Predicted: 1
[[0.95977928]]
0.9999999999999833
Predicted: 1
[[0.98938109]]
1.0
Predicted: 1
[[0.98779646]]
1.0
Predicted: 1
[[0.94797937]]
0.9999999999997524
Predicted: 1
[[0.9824625]]
1.0
Predicted: 1
[[0.97954435]]
1.0
Predicted: 1
[[0.99071802]]
1.0
Predicted: 1
[[0.95190642]]
0.9999999999998916
Predicted: 1
[[0.99270072]]
1.0
Predicted: 1
[[0.96547878]]
0.9999999999999967
Predicted: 1
[[0.99328332]]
1.0
Predicted: 1
[[0.95840961]]
0.9999999999999762
Predicted: 1
[[0.98250499]]
1.0
Predicted: 1
[[0.9

[[0.7909852]]
0.9999983400838093
Predicted: 1
[[0.87415987]]
0.9999999961780686
Predicted: 1
[[0.75471152]]
0.9999868490854616
Predicted: 1
[[0.70935759]]
0.9998666848035788
Predicted: 1
[[0.50478374]]
0.5476934477573047
Predicted: 1
[[0.70546069]]
0.9998390709027468
Predicted: 1
[[0.93134303]]
0.9999999999952602
Predicted: 1
[[0.89198897]]
0.9999999993222239
Predicted: 1
[[0.92203244]]
0.9999999999813072
Predicted: 1
[[0.93009685]]
0.9999999999942497
Predicted: 1
[[0.90957697]]
0.9999999999057252
Predicted: 1
[[0.95003483]]
0.9999999999998381
Predicted: 1
[[0.87167516]]
0.999999995218541
Predicted: 1
[[0.94260656]]
0.9999999999992997
Predicted: 1
[[0.90393833]]
0.9999999998162983
Predicted: 1
[[0.86614792]]
0.9999999922316206
Predicted: 1
[[0.74516075]]
0.9999781146275641
Predicted: 1
[[0.91229652]]
0.999999999932577
Predicted: 1
[[0.91648963]]
0.9999999999605427
Predicted: 1
[[0.92828775]]
0.9999999999924298
Predicted: 1
[[0.86051103]]
0.9999999874733666
Predicted: 1
[[0.94309581]]
0